# Data-analytiikka - Harjoitus 5

Huomaa, että tehtävät tulee palauttaa html-tiedostona, jossa näkyvät sekä koodisolujen sisällöt että tulosteet. Notebookin saat html-tiedostoksi valitsemalla File -> Download as -> HTML. Tehtävien vastaukset voi tehdä yhteen tai useampaan koodisoluun.

**Opiskelijan nimi**: Pyry Anttila

**Opiskelijanumero**: AD0828

## Aikaleimat - osa 1

Harjoituksen 2 tehtävässä 1.4 harjoiteltiin datan lukemista tekstitiedostosta (apache_log.txt). Luetussa datassa olevien muotoilujen takia aikaleimoja ei suoraan saatu samaan sarakkeeseen ja toivottuun muotoon. Jatketaan tässä tehtävässä aikaleiman käsittelyä luetusta datasta.

a) Lue data käyttöön joko harjoituksen 2 tavoin tai jos et saanut dataa tekstitiedostosta luettua, voit lukea sen csv:nä seuraavasta urlista. Lähtötilanne on alla olevan mallin mukainen:

![malli_apache1](https://student.labranet.jamk.fi/~ajota/Data-analytics/images/malli_apache_log_times_gmt_sep.png)

b) Muokkaa "Timestamp" ja "zone" sarakkeiden tiedoista hakasulkeet pois, ja muodosta sarakkeiden tiedoista yksi sarake "Timestamp + zone" (Älä lisää välilyöntiä aikaleiman ja aikavyöhyketiedon väliin).

c) Tiputa vanhat 'Timestamp' ja 'zone' -sarakkeet pois.

d) Muuta "Timestamp + zone"-sarakkeen tiedot Timestamp-objektiksi, utc=False.

e) Tee "Timestamp + zone" -sarakkeen avulla uusi "Timestamp (utc)" -sarake, missä aikaleimat ovat UTC-ajassa (Timestamp-objekteja).

In [1]:
import pandas as pd

In [2]:
# Solu / vastaus 1.1 a)-c)
# Kopioi Moodleen dataframen ensimmäinen rivi.
df = pd.read_csv('https://student.labranet.jamk.fi/~ajota/Datasets/apache_log_as_csv.csv', index_col=0)

df['Timestamp'] = df['Timestamp'].str.replace('[', '')
df['zone'] = df['zone'].str.replace(']', '')

df['Timestamp + zone'] = df['Timestamp'] + df['zone']

df = df.drop(columns=['Timestamp', 'zone'])


# Moodleen ensimmäinen rivi sarakeotsikoineen - siis koko tulostus.
df.head(1)

,Client's IP address,Identd,Userid,Request line from the client,Status code,Size of content,Referer,User-Agent HTTP request header,Timestamp + zone
0,109.169.248.247,-,-,GET /administrator/ HTTP/1.1,200,4263,-,Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20...,12/Dec/2015:18:25:11+0100


In [3]:
# Solu / vastaus 1.1 d)
import pytz

date_format = "%d/%b/%Y:%H:%M:%S%z"

df['Timestamp + zone'] = pd.to_datetime(df['Timestamp + zone'], format=date_format, utc=False)

offset = pytz.FixedOffset(60)

df['Timestamp + zone'] = df['Timestamp + zone'].dt.tz_convert(offset)

df['Timestamp + zone'][0]

Timestamp('2015-12-12 18:25:11+0100', tz='pytz.FixedOffset(60)')

In [4]:
# Solu / vastaus 1.1 e)

df['Timestamp (utc)'] = df['Timestamp + zone'].dt.tz_convert('UTC')

df['Timestamp (utc)'][0]

Timestamp('2015-12-12 17:25:11+0000', tz='UTC')

## Aikaleimat - osa 2

Toisinaan dataseteissä aika ilmoitetaan Unix-aikaleimana, missä aika on ilmoitettu sekunteina ajanhetkestä 1.1.1970 0:00:00 UTC lähtien.

Tämän harjoituksen tavoitteena on opetella muuttamaan unix-aika Pandasin Timestampiksi helpommin luettavaan ja käsiteltävään muotoon.

tehdään ryhmittelyä esim. päivätasolla -> pitää saada esille päivä ja tehdä tästä uusi sarake

a) Lue 'normal_traffic_unsw_nb15.csv' -data dataframeen osoitteesta 'https://student.labranet.jamk.fi/~ajota/Datasets/normal_traffic_unsw_nb15.csv'.

b) Datan sarakkeissa on tieto 'record start time'. Tee uusi sarake 'Timestamp', missä sarakkeen 'record start time' on muutettu unix-ajasta Pandasin Timestampiksi. Tehdään tässä vaiheessa aikaleimoista tz-naiiveja. Älä siis vielä lisää tietoa aikavyöhykkeestä. Järjestä dataframen tiedot aikaleiman mukaan nousevaan järjestykseen (aikaleiman mukaan ensimmäinen tieto ensimmäisellä rivillä), käytä järjestäessä inplace=True ja aseta indeksi uudelleen. Tulosta näkyviin sarakkeen ensimmäinen tieto.

c) Tee uudet sarakkeet 'Year', 'Month', 'Day' ja 'Hour', joihin erotat 'Timestamp'-sarakkeesta vastaavat vuosi, kuukausi, päivä ja tunti -tiedot. (Huomaa että 'Timestamp'-sarakkeen tiedot ovat (tai pitäisi olla!) Pandasin Timestamp-objekteja. Vilkaise esim. Pandasin dokumentaatiosta mitä attribuutteja Timestamp-objektilla on, jotta saat tiedot näppärästi muodostettua.) Tulosta dataframen ensimmäiseltä riviltä sarakkeiden 'Timestamp', 'Year', 'Month', 'Day' ja 'Hour' -tiedot näkyviin.

d) Tee uusi sarake 'Timestamp Canberra', missä on 'Timestamp'-sarakkeen tiedot lokalisoituina Australian Canberran aikavyhykkeelle.

e) Tee uusi sarake 'Timestamp Helsinki', missä on 'Timestamp Canberra' -sarakkeen tiedot muutettu Helsingin aikavyöhykkeelle.

f) Laske Hour-sarakkeen avulla, miltä tunnilta datassa on vähiten rivejä / tietoja.

In [5]:
# Luetaan data
dfn = pd.read_csv('https://student.labranet.jamk.fi/~ajota/Datasets/normal_traffic_unsw_nb15.csv', index_col=0)

In [6]:
# Solu / vastaus 1.2 b)

dfn['Timestamp'] = pd.to_datetime(dfn['record start time'], unit='s')

dfn.sort_values(by='Timestamp', inplace=True)

dfn.reset_index(drop=True, inplace=True)

# Moodleen tämän tulostus:
dfn['Timestamp'][0]

Timestamp('2015-01-22 11:50:43')

In [7]:
# Solu / vastaus 1.2 c)

dfn['Year'] = dfn['Timestamp'].dt.year
dfn['Month'] = dfn['Timestamp'].dt.month
dfn['Day'] = dfn['Timestamp'].dt.day
dfn['Hour'] = dfn['Timestamp'].dt.hour

# Moodleen tämä tulostus:
dfn[['Timestamp', 'Year', 'Month', 'Day', 'Hour']].head(1)

,Timestamp,Year,Month,Day,Hour
0,2015-01-22 11:50:43,2015,1,22,11


In [8]:
# Solu / vastaus 1.2 d)

dfn['Timestamp Canberra'] = dfn['Timestamp'].dt.tz_localize('UTC').dt.tz_convert('Australia/Canberra')

dfn['Timestamp Canberra'][0]

Timestamp('2015-01-22 22:50:43+1100', tz='Australia/Canberra')

In [9]:
# Solu / vastaus 1.2 e)

dfn['Timestamp Helsinki'] = dfn['Timestamp Canberra'].dt.tz_convert('Europe/Helsinki')

dfn['Timestamp Helsinki'][0]

Timestamp('2015-01-22 13:50:43+0200', tz='Europe/Helsinki')

In [10]:
# Solu / vastaus 1.2 f)
# Miltä tunnilta on vähiten tietoja? Kirjoita Moodleen ko. tunti.

hcount = dfn['Hour'].value_counts()
minh = hcount.idxmin()

print(minh)

11


## Aikaleimat - osa 3

Käytetään seuraavissa tehtävissä edellisen kohdan dataa. Selvitä vastaukset seuraaviin kysymyksiin kohdat uudelleen näytteistämällä dataa.

a) Mistä IP-osoitteesta oli eniten yhteydenottoja tuntitasolla?

b) Milloin datassa näkyy eniten yhteydenottoja (rivejä), kun dataa tarkastellaan 30 minuutin jaksoissa? Käytä tiedon selvittämiseen apuna 'Timestamp'-saraketta ja resample-funktiota.

In [11]:
# Solu / vastaus 1.3 a)
# Kirjoita Moodleen 'Source IP address', josta on ollut eniten yhteydenottoja tuntitasolla.

ip_addr = dfn.groupby(['Source IP address', 'Hour']).size().idxmax()[0]
print(ip_addr)

59.166.0.8


In [12]:
# Solu / vastaus 1.3 b)
# Kirjoita Moodleen aikaleima, jolloin yhteydenottoja on ollut eniten. (Itse asiassa kyseessä aikaväli, mistä koosteessa näkyy vain aikavälin ensimmäinen arvo. Puolen tunnin välein näytteistettynä ensimmäiset ajat ovat 2015-01-22 11:30:00, 2015-01-22 12:00:00, 2015-01-22 12:30:00, ...)

dfn.set_index('Timestamp', inplace=True)

rdata = dfn.resample('30min').size()

maxt = rdata.idxmax()

print(maxt)

2015-01-22 16:30:00
